In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from collections import Counter

In [ ]:

train_df = pd.read_csv('/content/drive/MyDrive/Train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Test.csv')
train_df.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S23,S24,S25,S26,S27,S28,S29,S30,??,Label
0,$ 0.124591$,& 00@36537%,`]0^10:2904,% 10h100042154%,☢️ 0024 2 💀2104☢️,$ 0.122790$,& 01@19994%,[~ ~|%0|162744,%0S0 0269440%,☢️ 9 🤖90134813☢️,...,">\^& :%""0626380",% 070207n752%,⚠️ 8 💀81041476⚠️,$ 0.726284$,& 08@58165%,/ ]?*;%*0864882,% 053832h294%,⚠️ 0 10 🤡10296524⚠️,🙂,R
1,$ 0.015186$,& 00@28209%,", $`'`]0071861",%1C1 0206235%,⚠️ 01756 0 🤖098⚠️,$ 0.108401$,& 01@47976%,[ :?0|{~136437,& 0Y00218220&,⚠️-000689 0 🤖06⚠️,...,"""*) 09'&20850",& 0987265j55&,⚠️ 08156 2 💀242⚠️,$ 0.684961$,& 04@35579%,&~~[ _`:0550468,% 05653S3769%,☢️ 01 6 🤡665668☢️,🙂,R
2,$-0.005414$,& 00@40202%,'0]*_[/008480,& 0077341r15&,☢️ 00594 6 👾660☢️,$-0.008421$,& 02@34221%,<];$ '0144866,& 04016K6501&,⚠️ 0075 9 💀9857⚠️,...,"_* <[""'""0050316",& 019409E976&,⚠️-027 9 🤖96671⚠️,$ 1.040288$,& 08@15316%,#@} 07>01916,% 084W432679%,☢️ 022 3 🤖37419☢️,🙂,M
3,$ 0.007761$,& 00@07632%,"= *$}:0""042436",& 07j7046098&,☢️ 0 6 🤡6025390☢️,$-0.047624$,& 00@58905%,_%* @&0!196483,% 017186W698%,⚠️ 007788 10 👾103⚠️,...,"-0`,03_{3549",& 06810P107553&,☢️ 0662 0 🤖0205☢️,$ 1.012432$,&-01@17631%,=>>{=? 0942173,%7W7 0932095%,☢️ 07811 7 🤡759☢️,🙂,M
4,$ 0.008367$,& 00@96862%,0]=\031994,& 0062o21953&,⚠️ 3 👾30108433⚠️,$-0.000739$,& 01@28325%,<&~ =0{%235521,%0X0 0291510%,☢️ 01 1 👾160132☢️,...,:' 1(139496,&6X6 0936152&,⚠️ 067828 5 💀59⚠️,$ 0.990962$,& 10@23887%,=$$07'71772,%10m10 0852411%,☢️ 0253 8 🤖8410☢️,🙂,M


In [ ]:
# Dropping 'Label' and '??' from X_train in the training dataset
X_train = train_df.drop(['Label', '??'], axis=1)  # Adjust '??' to the actual column name if known
y_train = train_df['Label'].replace({'R': 0, 'M': 1})  # Convert 'R' and 'M' to 0 and 1


# Display the first few rows of the cleaned datasets
print("Cleaned X_train:")
print(X_train.head())

Cleaned X_train:
            S1           S2              S3               S4  \
0  $ 0.124591$  & 00@36537%     `]0^10:2904  % 10h100042154%   
1  $ 0.015186$  & 00@28209%  , $`'`]0071861    %1C1 0206235%   
2  $-0.005414$  & 00@40202%   '0]*_[/008480    & 0077341r15&   
3  $ 0.007761$  & 00@07632%  = *$}:0"042436    & 07j7046098&   
4  $ 0.008367$  & 00@96862%      0]=\031994    & 0062o21953&   

                  S5           S6           S7              S8             S9  \
0  ☢️ 0024 2 💀2104☢️  $ 0.122790$  & 01@19994%  [~ ~|%0|162744  %0S0 0269440%   
1  ⚠️ 01756 0 🤖098⚠️  $ 0.108401$  & 01@47976%  [ :?0|{~136437  & 0Y00218220&   
2  ☢️ 00594 6 👾660☢️  $-0.008421$  & 02@34221%   <];$ '0144866  & 04016K6501&   
3  ☢️ 0 6 🤡6025390☢️  $-0.047624$  & 00@58905%  _%* @&0!196483  % 017186W698%   
4  ⚠️  3 👾30108433⚠️  $-0.000739$  & 01@28325%  <&~ =0{%235521  %0X0 0291510%   

                   S10  ...          S21          S22              S23  \
0    ☢️  9 🤖90134813☢️  ...  $-0.0003

<ipython-input-5-302412bfe264>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = train_df['Label'].replace({'R': 0, 'M': 1})  # Convert 'R' and 'M' to 0 and 1


In [ ]:
# Load the test dataset
test_df = pd.read_csv('/content/drive/MyDrive/Test.csv')

# Define the cleaning function (as you already have)
import re

def clean_and_process_cell(cell):
    # Remove all characters except for '-' and digits
    filtered = re.sub(r'[^0-9\-]', '', str(cell))

    # If filtered string is empty, return 0.0
    if not filtered:
        return 0.0

    # Handle negative numbers
    is_negative = filtered.startswith('-')
    filtered = filtered.lstrip('-')

    # Add decimal point after one digit from the left
    if len(filtered) > 1:
        cleaned_value = f"{filtered[0]}.{filtered[1:]}"
    else:
        cleaned_value = f"{filtered[0]}.0"

    # Convert cleaned value to float
    try:
        number = float(cleaned_value)
    except ValueError:
        return 0.0  # Return 0.0 if conversion fails

    # Apply negative sign if necessary
    if is_negative:
        number = -number

    # If the number is greater than 1, divide by 10
    if number > 1:
        number /= 10

    return number

# Apply the clean_and_process_cell function to both X_train and test_df
X_train = X_train.applymap(clean_and_process_cell)
test_df = test_df.applymap(clean_and_process_cell)

# Show the processed DataFrames
print("Cleaned X_train:")
print(X_train.head())

print("\nCleaned Test DataFrame (test_df):")
print(test_df.head())

<ipython-input-6-54f6e2cf6f94>:42: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_train = X_train.applymap(clean_and_process_cell)
<ipython-input-6-54f6e2cf6f94>:43: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_df = test_df.applymap(clean_and_process_cell)


Cleaned X_train:
         S1        S2        S3        S4        S5        S6        S7  \
0  0.124591  0.036537  0.102904  0.101000  0.024221  0.122790  0.119994   
1  0.015186  0.028209  0.071861  0.110206  0.175601  0.108401  0.147976   
2 -0.005414  0.040202  0.008480  0.077341  0.059467 -0.008421  0.234221   
3  0.007761  0.007632  0.042436  0.770461  0.660254 -0.047624  0.058905   
4  0.008367  0.096862  0.031994  0.062220  0.330108 -0.000739  0.128325   

         S8        S9       S10  ...       S21       S22       S23       S24  \
0  0.162744  0.002694  0.990135  ... -0.000319  0.258920  0.626380  0.702078   
1  0.136437  0.002182 -0.006890  ...  0.256636  0.624720  0.920850  0.987266   
2  0.144866  0.401665  0.075999  ... -0.222064  0.104655  0.050316  0.194100   
3  0.196483  0.171867  0.077881  ...  0.524351  0.406711 -0.033549  0.681011   
4  0.235521  0.002915  0.111601  ...  0.809183  0.587020  0.113950  0.660936   

        S25       S26       S27       S28       S29

In [ ]:
# Required Libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Assuming your X_train and y_train are already prepared
# X_train = Features, y_train = Target variable (binary classification)

# Split the data into training (80%) and testing sets (20%)
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Initialize XGBoost classifier with tuned parameters
xgb_model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    learning_rate=0.05,  # Try decreasing to 0.05 or lower for a more gradual learning process
    n_estimators=400,  # Number of trees to build, increase for better results but longer training
    max_depth=7,  # Control tree complexity; increase if underfitting
    subsample=0.8,  # Use a fraction of samples for training each tree
    colsample_bytree=0.8,  # Use a fraction of features for training
    gamma=0.1,  # Regularization to prevent overfitting, try increasing this
    reg_lambda=1,  # L2 regularization
    reg_alpha=0,  # L1 regularization, increase for sparse features
)

# Train the XGBoost model
xgb_model.fit(X_train_split, y_train_split)

# Make predictions on the test data
y_pred_test = xgb_model.predict(X_test_split)

# Check the accuracy on the test data
accuracy_test = accuracy_score(y_test_split, y_pred_test)
print(f"Test accuracy: {accuracy_test * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:49:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test accuracy: 73.06%


In [ ]:
# Required Libraries
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Assuming your X_train (features) and y_train (labels) are already prepared

# Split data into train and test sets (80% train, 20% test)
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Initialize LightGBM classifier with tuned parameters
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=50,
    max_depth=-1,
    learning_rate=0.05,
    n_estimators=500,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# Train the model
lgb_model.fit(X_train_split, y_train_split)

# Make predictions on the test data
y_pred_test = lgb_model.predict(X_test_split)

# Calculate accuracy
accuracy_test = accuracy_score(y_test_split, y_pred_test)
print(f"Test accuracy: {accuracy_test * 100:.2f}%")




[LightGBM] [Info] Number of positive: 28337, number of negative: 27663
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018644 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7650
[LightGBM] [Info] Number of data points in the train set: 56000, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506018 -> initscore=0.024073
[LightGBM] [Info] Start training from score 0.024073
Test accuracy: 73.51%


In [ ]:
X_test = test_df.iloc[:, 1:31]  # Select columns 1 to 30 (0-based indexing)

# Step 3: Make predictions using the trained model
y_pred_final = lgb_model.predict(X_test)

# Step 4: Map predictions from 0/1 to 'R'/'M'
y_pred_final = pd.Series(y_pred_final).replace({0: 'R', 1: 'M'})

# Step 5: Create a results DataFrame with sequential IDs and predicted Label
results = pd.DataFrame({
    'Id': range(len(y_pred_final)),  # Sequential IDs starting from 0
    'Label': y_pred_final  # Include the predicted labels
})

# Save the results to a CSV file
results.to_csv('predictionslgb.csv', index=False)

print("Predictions saved to predictionslgb.csv.")

Predictions saved to predictionslgb.csv.
